# Airbus Hackatuna

In [1]:
# =====================================================
# INITIALISATION ENVIRONNEMENT PROJET (PROPRE)
# =====================================================

import os
import sys
import warnings
import importlib
import csv
import numpy as np

from tqdm import tqdm

import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import sys
sys.path.append("../")
sys.path.append("../src")

import config as c
import utils_N as u
import network_N as n
import lidar_utils

warnings.filterwarnings("ignore")

In [ ]:
# # =====================================================
# # INITIALISATION ENVIRONNEMENT PROJET (PROPRE)
# # =====================================================
# 
# import os
# import sys
# import warnings
# import importlib
# import csv
# import numpy as np
# 
# from tqdm import tqdm
# 
# import h5py
# 
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# 
# import os
# import sys
# 
# import os
# import sys
# 
# warnings.filterwarnings("ignore")
# 
# # === Chemins absolus projet ===
# PROJECT_ROOT = r"C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main"
# SRC_PATH = os.path.join(PROJECT_ROOT, "src")
# 
# # === Nettoyage modules potentiellement mal chargés ===
# for module in ["config", "utils_N", "lidar_utils", "network"]:
#     if module in sys.modules:
#         del sys.modules[module]
# 
# # === Injection chemins prioritaires ===
# if PROJECT_ROOT not in sys.path:
#     sys.path.insert(0, PROJECT_ROOT)
# 
# if SRC_PATH not in sys.path:
#     sys.path.insert(0, SRC_PATH)
# 
# # =====================================================
# # IMPORTS PROJET
# # =====================================================
# 
# import config
# import utils_N
# import lidar_utils
# import network
# 
# # Reload pour sécurité
# importlib.reload(config)
# importlib.reload(utils_N)
# importlib.reload(lidar_utils)
# importlib.reload(network)
# 
# # Alias propres
# c = config
# u = utils_N
# lu = lidar_utils
# n = network
# 
# # =====================================================
# # VERIFICATION DES FICHIERS CHARGÉS
# # =====================================================
# 
# print("CONFIG FILE →", c.__file__)
# print("UTILS FILE  →", u.__file__)
# print("LIDAR FILE  →", lu.__file__)
# print("NETWORK FILE→", n.__file__)
# 
# print("\nDataset class →", n.PointCloudDataset)
# print("Model class   →", n.PointNetSeg)


CONFIG FILE → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\config.py
UTILS FILE  → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\utils_N.py
LIDAR FILE  → C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\lidar_utils.py
NETWORK FILE→ C:\Users\Léopold\Downloads\Airbus-Hackathon-2026-main\Airbus-Hackathon-2026-main\src\network.py

Dataset class → <class 'network.PointCloudDataset'>
Model class   → <class 'network.PointNetSeg'>


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = n.PointNetSeg(num_classes = 4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
weights = torch.tensor([1.0, 8.0, 1.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

In [ ]:
# Charger poids sauvegardés
model_path = "../models/PointNetSeg_cpu.pth"  # adapte si besoin

model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle chargé et en mode évaluation")
test_ds = n.PointCloudDataset("../datasets/processed/test.h5")
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)

print("✅ Test loader prêt")

✅ Modèle chargé et en mode évaluation
✅ Test loader prêt


In [7]:
def predict_segmentation(model, dataloader, device):

    model.eval()

    all_preds = []
    all_points = []
    all_poses = []

    with torch.no_grad():
        for points, labels, poses in dataloader:

            points = points.to(device)

            logits = model(points)
            preds = torch.argmax(logits, dim=2)

            all_preds.extend(preds.cpu().numpy())
            all_points.extend(points.cpu().numpy())
            all_poses.extend(poses.cpu().numpy())

    return all_preds, all_points, all_poses

In [8]:
import open3d as o3d
import numpy as np

def cluster_by_class(frame_points,
                     frame_preds,
                     class_id):

    # =========================
    # PARAMÈTRES PAR CLASSE
    # =========================
    if class_id == 1:        # Cable
        eps = 0.6
        min_points = 8
    elif class_id == 2:      # Electric pole
        eps = 0.8
        min_points = 15
    elif class_id == 3:      # Wind turbine
        eps = 1.2
        min_points = 25
    else:                    # Antenna
        eps = 0.7
        min_points = 10

    # =========================
    # FILTRER CLASSE
    # =========================
    mask = frame_preds == class_id
    class_points = frame_points[mask]

    if len(class_points) < min_points:
        return []

    # Garder XYZ uniquement
    class_points = class_points[:, :3].astype(np.float64)

    # =========================
    # SUPPRESSION SOL SIMPLE
    # =========================
    # On enlève les points trop proches du sol
    z_threshold = np.percentile(class_points[:, 2], 10)
    class_points = class_points[class_points[:, 2] > z_threshold]

    if len(class_points) < min_points:
        return []

    # =========================
    # DBSCAN
    # =========================
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(class_points)

    labels = np.array(
        pcd.cluster_dbscan(
            eps=eps,
            min_points=min_points,
            print_progress=False
        )
    )

    clusters = []

    for k in np.unique(labels):
        if k == -1:
            continue

        cluster_pts = class_points[labels == k]

        if len(cluster_pts) < min_points:
            continue

        clusters.append(cluster_pts)

    return clusters

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
def compute_obb(cluster_pts):

    if cluster_pts.shape[0] < 5:
        return None

    pts = cluster_pts[:, :3].astype(np.float64)

    # =========================
    # 1️⃣ PCA SUR XY
    # =========================
    xy = pts[:, :2]

    mean_xy = np.mean(xy, axis=0)
    xy_centered = xy - mean_xy

    cov = np.cov(xy_centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)

    order = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, order]

    R2 = eigvecs

    # =========================
    # 2️⃣ PROJECTION + FILTRE OUTLIERS
    # =========================
    proj = xy_centered @ R2

    # IMPORTANT : suppression outliers
    min_xy = np.percentile(proj, 2, axis=0)
    max_xy = np.percentile(proj, 98, axis=0)

    extent_xy = max_xy - min_xy
    center_xy_pca = (min_xy + max_xy) / 2

    center_xy = mean_xy + center_xy_pca @ R2.T

    # =========================
    # 3️⃣ Z SEPARE
    # =========================
    min_z = np.percentile(pts[:, 2], 2)
    max_z = np.percentile(pts[:, 2], 98)

    center_z = (min_z + max_z) / 2
    extent_z = max_z - min_z

    center = np.array([center_xy[0], center_xy[1], center_z])
    extent = np.array([extent_xy[0], extent_xy[1], extent_z])

    yaw = np.arctan2(R2[1, 0], R2[0, 0])

    return {
        "center": center,
        "extent": extent,
        "yaw": yaw
    }


In [10]:
def full_inference_pipeline(model, dataloader, device):

    preds, points, poses = predict_segmentation(model, dataloader, device)

    detections = []

    for i in range(len(points)):

        frame_points = points[i]
        frame_preds  = preds[i]
        frame_pose   = poses[i]

        for class_id in [0,1,2,3]:

            clusters = cluster_by_class(
                frame_points,
                frame_preds,
                class_id
            )

            for cluster_pts in clusters:

                obb = compute_obb(cluster_pts)

                if obb is None:
                    continue

                detections.append({
                    "ego": frame_pose,
                    "center": obb["center"],
                    "extent": obb["extent"],
                    "yaw": obb["yaw"],
                    "class_id": class_id
                })

    return detections

In [12]:
detections = full_inference_pipeline(model, test_loader, device)

print("Nombre total de bounding boxes détectées :", len(detections))

Nombre total de bounding boxes détectées : 5174


In [13]:
def full_gt_pipeline(dataloader):

    detections_gt = []

    for points, labels, poses in dataloader:

        points = points.numpy()
        labels = labels.numpy()
        poses  = poses.numpy()

        for i in range(len(points)):

            frame_points = points[i]
            frame_labels = labels[i]
            frame_pose   = poses[i]

            for class_id in [0,1,2,3]:

                clusters = cluster_by_class(
                    frame_points,
                    frame_labels,
                    class_id
                )

                for cluster_pts in clusters:

                    obb = compute_obb(cluster_pts)

                    if obb is None:
                        continue

                    detections_gt.append({
                        "ego": frame_pose,
                        "center": obb["center"],
                        "extent": obb["extent"],
                        "yaw": obb["yaw"],
                        "class_id": class_id
                    })

    return detections_gt

In [15]:
from shapely.geometry import Polygon

def compute_iou_3d(box1, box2):

    c1, e1, yaw1 = box1["center"], box1["extent"], box1["yaw"]
    c2, e2, yaw2 = box2["center"], box2["extent"], box2["yaw"]

    def get_corners(center, extent, yaw):
        dx, dy = extent[0] / 2, extent[1] / 2

        corners = np.array([
            [-dx, -dy],
            [ dx, -dy],
            [ dx,  dy],
            [-dx,  dy]
        ])

        R = np.array([
            [np.cos(yaw), -np.sin(yaw)],
            [np.sin(yaw),  np.cos(yaw)]
        ])

        rotated = corners @ R.T
        return rotated + center[:2]

    poly1 = Polygon(get_corners(c1, e1, yaw1))
    poly2 = Polygon(get_corners(c2, e2, yaw2))

    if not poly1.is_valid or not poly2.is_valid:
        return 0.0

    inter_area = poly1.intersection(poly2).area
    union_area = poly1.area + poly2.area - inter_area

    if union_area == 0:
        return 0.0

    z1_min = c1[2] - e1[2] / 2
    z1_max = c1[2] + e1[2] / 2

    z2_min = c2[2] - e2[2] / 2
    z2_max = c2[2] + e2[2] / 2

    inter_z = max(0, min(z1_max, z2_max) - max(z1_min, z2_min))

    inter_vol = inter_area * inter_z
    vol1 = poly1.area * e1[2]
    vol2 = poly2.area * e2[2]

    union_vol = vol1 + vol2 - inter_vol

    if union_vol == 0:
        return 0.0

    return inter_vol / union_vol


In [16]:
def evaluate_object_iou(detections_pred, detections_gt, threshold=0.5):

    ious = []
    matches = 0

    for pred in detections_pred:

        best_iou = 0

        for gt in detections_gt:

            if not np.allclose(pred["ego"], gt["ego"]):
                continue

            if pred["class_id"] != gt["class_id"]:
                continue

            iou = compute_iou_3d(pred, gt)

            if iou > best_iou:
                best_iou = iou

        ious.append(best_iou)

        if best_iou > threshold:
            matches += 1

    mean_iou = np.mean(ious) if len(ious) > 0 else 0

    print("Mean Object IoU :", mean_iou)
    print("Correct detections (IoU>0.5):", matches)

    return mean_iou

In [17]:
detections_pred = full_inference_pipeline(model, test_loader, device)
print("Pred boxes:", len(detections_pred))

detections_gt = full_gt_pipeline(test_loader)
print("GT boxes:", len(detections_gt))

mean_iou = evaluate_object_iou(detections_pred, detections_gt)


Pred boxes: 5174
GT boxes: 6629
Mean Object IoU : 0.29484262121191573
Correct detections (IoU>0.5): 1523


In [18]:
_, all_points, _ = predict_segmentation(model, test_loader, device)

In [19]:
def visualize_frame_comparison(frame_index,
                                points_list,
                                detections_pred,
                                detections_gt):

    frame_points = points_list[frame_index][:, :3]
    frame_pose = detections_gt[frame_index]["ego"]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(frame_points)
    pcd.paint_uniform_color([0.6, 0.6, 0.6])

    geometries = [pcd]

    # PRED (ROUGE)
    for det in detections_pred:
        if not np.allclose(det["ego"], frame_pose):
            continue

        obb = o3d.geometry.OrientedBoundingBox(
            center=det["center"],
            R=o3d.geometry.get_rotation_matrix_from_axis_angle([0, 0, det["yaw"]]),
            extent=det["extent"]
        )
        obb.color = (1, 0, 0)
        geometries.append(obb)

    # GT (VERT)
    for det in detections_gt:
        if not np.allclose(det["ego"], frame_pose):
            continue

        obb = o3d.geometry.OrientedBoundingBox(
            center=det["center"],
            R=o3d.geometry.get_rotation_matrix_from_axis_angle([0, 0, det["yaw"]]),
            extent=det["extent"]
        )
        obb.color = (0, 1, 0)
        geometries.append(obb)

    o3d.visualization.draw_geometries(geometries)


In [ ]:
visualize_frame_comparison(
    frame_index=0,
    points_list=all_points,
    detections_pred=detections_pred,
    detections_gt=detections_gt
)

In [21]:
import csv
import numpy as np

CLASS_LABELS = {
    0: "Antenna",
    1: "Cable",
    2: "Electric Pole",
    3: "Wind Turbine"
}

def export_detections_to_csv(detections, output_csv_path):
    """
    Exporte les détections au format demandé par Airbus.

    Colonnes:
      ego_x, ego_y, ego_z, ego_yaw (copiés tels quels depuis 'ego')
      bbox_center_x/y/z (m)
      bbox_width, bbox_length, bbox_height (m)  -> dimensions dans le repère de la box AVANT yaw
      bbox_yaw (rad) (rotation autour Z)
      Class ID
      Class Label
    """

    header = [
        "ego_x", "ego_y", "ego_z", "ego_yaw",
        "bbox_center_x", "bbox_center_y", "bbox_center_z",
        "bbox_width", "bbox_length", "bbox_height",
        "bbox_yaw",
        "Class ID", "Class Label"
    ]

    with open(output_csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

        for det in detections:
            ego = det["ego"]
            ego = np.array(ego).reshape(-1)

            # Sécurité: on prend les 4 premières composantes si plus long
            if ego.shape[0] < 4:
                # si jamais ego ne contient pas 4 valeurs, on skip
                continue
            ego_x, ego_y, ego_z, ego_yaw = ego[0], ego[1], ego[2], ego[3]

            center = det["center"]
            extent = det["extent"]
            yaw = det["yaw"]
            class_id = int(det["class_id"])

            bbox_center_x = float(center[0])
            bbox_center_y = float(center[1])
            bbox_center_z = float(center[2])

            # IMPORTANT:
            # - extent[0] = dimension le long de l’axe principal de la box (x' avant yaw)
            # - extent[1] = dimension le long de l’axe secondaire (y' avant yaw)
            # Airbus demande width/length along x,y before yaw -> on map:
            bbox_length = float(extent[0])  # x' (axe principal)
            bbox_width  = float(extent[1])  # y'
            bbox_height = float(extent[2])  # z

            class_label = CLASS_LABELS.get(class_id, "Unknown")

            writer.writerow([
                ego_x, ego_y, ego_z, ego_yaw,
                bbox_center_x, bbox_center_y, bbox_center_z,
                bbox_width, bbox_length, bbox_height,
                float(yaw),
                class_id, class_label
            ])

    print(f"✅ CSV exporté : {output_csv_path}")


In [23]:
# Export CSV (prédictions)
export_detections_to_csv(
    detections_pred,
    output_csv_path= "../outputs/CSVs/test_predictions.csv"
)

✅ CSV exporté : ../outputs/CSVs/test_predictions.csv
